In [80]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
import os
import json
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv(override=True)

True

In [81]:
import subprocess
import socket

# Get Windows host IP from WSL
def get_windows_host_ip():
    # Method 1: Try WSL2 bridge interface (default route)
    try:
        result = subprocess.run(
            ["ip", "route", "show", "default"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'default' in line and 'via' in line:
                parts = line.split()
                via_index = parts.index('via')
                ip = parts[via_index + 1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (WSL default gateway)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 1 failed: {e}")
    
    # Method 2: Try /etc/resolv.conf nameserver
    try:
        result = subprocess.run(
            ["cat", "/etc/resolv.conf"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'nameserver' in line:
                ip = line.split()[1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (resolv.conf)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 2 failed: {e}")
    
    # Fallback to localhost
    print("⚠ Falling back to localhost")
    return "127.0.0.1"
    
WINDOWS_HOST_IP = get_windows_host_ip()

✓ Found Ollama at 172.17.96.1 (WSL default gateway)


In [82]:
filename_translation = {}

for folder in os.listdir("./assets/input"):
    for file in os.listdir(f"./assets/input/{folder}"):
        base_name = os.path.splitext(file)[0]
        filename_translation[base_name] = ""

with open(file="./filename_translation.json", mode="w", encoding="utf-8") as f:
    json.dump(filename_translation, f, ensure_ascii=False, indent=2)

In [77]:
# llm = ChatOllama(base_url=f"http://{WINDOWS_HOST_IP}:11434",
#                  model="qwen3:1.7b",
#                  temperature=0,
#                  max_tokens=128)

GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
llm = ChatOpenAI(
    model="openai/gpt-4.1-mini",
    temperature=0,
    base_url="https://models.github.ai/inference",
    api_key=GITHUB_TOKEN,
)

In [78]:
def ch_to_eng(text: str) -> str:
    """Translate Chinese to English"""
    system = SystemMessage(
        content="You are a Mandarin to English translator. Only translate without adding or removing anything"
    )
    human = HumanMessage(content=text)
    resp = llm.invoke(input=[system, human])
    return resp.content

In [73]:
for filename in list(filename_translation):
    if filename_translation[filename] == "":
        translated_filename = ch_to_eng(filename)
        filename_translation[filename] = translated_filename
        print(f"Translated {filename} to {translated_filename}")

        with open(file="./filename_translation.json", mode="w", encoding="utf-8") as f:
            json.dump(filename_translation, f, ensure_ascii=False, indent=2)

Translated HA-4-0005(7)門診全身手術麻醉後注意事項_0.md to HA-4-0005(7) Outpatient General Surgery Post-Anesthesia Care Instructions_0.md
Translated HA-4-0009(1)認識硬脊膜外自控式減痛_0.md to HA-4-0009(1) Understanding Epidural Patient-Controlled Analgesia_0.md
Translated HA-4-0011侵入性檢查與治療麻醉注意事項_0.md to HA-4-0011 Invasive Examination and Treatment Anesthesia Precautions_0.md
Translated HA-4-0010(1)認識病人靜脈自控式減痛_0.md to HA-4-0010(1) Understanding Patient-Controlled Intravenous Analgesia_0.md
Translated HA-7-0003(1)_八字肩帶使用注意事項_0.md to HA-7-0003(1)_Eight-Character Shoulder Strap Usage Precautions_0.md
Translated HA-7-0042(1)_胸部挫傷注意事項_0.md to HA-7-0042(1)_Chest Contusion Precautions_0.md
Translated HA-7-0020(1)_兒童上呼吸道感染注意_0.md to HA-7-0020(1)_Precautions for Upper Respiratory Tract Infection in Children_0.md
Translated HA-7-0037(1)_流鼻血注意事項_0.md to HA-7-0037(1)_Nosebleed Precautions_0.md
Translated HA-7-0036(1)_急性腸胃炎注意事項_0.md to HA-7-0036(1)_Precautions for Acute Gastroenteritis_0.md
Translated HA-7-0026(1)_兒童嘔吐及腹瀉注意

KeyboardInterrupt: 